<a href="https://www.kaggle.com/code/nadaarfaoui/web-summarizer?scriptVersionId=288507270" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install requests beautifulsoup4 python-dotenv huggingface_hub

In [2]:
from huggingface_hub import InferenceClient
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import requests
import os

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HUGGING_FACE_TOKEN")

In [4]:
# conversational-capable model
client = InferenceClient(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    token=secret_value_0
)

# Fetch and clean a webpage
headers = {"User-Agent": "Mozilla/5.0"}
class Website:
    def __init__(self, url):
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for tag in soup(["script", "style", "img", "input"]):
            tag.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [5]:
def summarize(url):
    website = Website(url)

    system_prompt = (
        "You are an assistant that analyzes the contents of a website "
        "and provides a short summary, ignoring navigation or repetitive elements. "
        "Respond in markdown."
    )

    user_prompt = f"""
Website title: {website.title}
Content:
{website.text[:4000]}
"""

    response = client.chat_completion(
        model="mistralai/Mistral-7B-Instruct-v0.2",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=500,
    )

    return response.choices[0].message["content"]

In [6]:
# Display function
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

# Test it
display_summary("https://edwarddonner.com")

 # Edward Donner - Home

Edward Donner's homepage introduces him as a tech enthusiast and CTO of Nebula.io, where they use AI to help people discover their potential. Donner also shares his interests in coding, DJing, electronic music production, and Hacker News. He is a co-founder and a previous CEO of an acquired AI startup, untapt. Nebula.io possesses unique LLMs, a patented matching model, and an award-winning platform. Visitors can connect with Donner via email or social media links provided. The site has a blog section, named 'Posts', with articles on topics like AI in production and being an AI engineer. There's an option to subscribe to his newsletter.